In [1]:
using LinearAlgebra

In [ ]:
function Taylor(funcão, derivadas, referência, desejado)  #TODO
    grau=len(derivadas) #grau do Taylor vai ser implícito
    return função(desejado)
end

In [ ]:
function resolver_equações(função,d,precisão)
    return solução
end

In [ ]:
function achar_zero_de_função(função,precisão) #intervalo?, derivadas?
    return solução
end

In [ ]:
function bisseção(função, precisão, intervalo) #Feito
    return zero_da_função
end

In [ ]:
function ponto_fixo(função,precisão,chute_inicial) #Feito 
    return ponto_fixo_da_função
end

In [ ]:
function newton(função,derivada, precisão, chute_inicial) #Feito
    return zero_da_função
end

In [ ]:
function resolver_equaçoes_diferenciais()
    return 
end

In [ ]:
function resolver_PVI(derivada(x,y),x_inicial,y_inicial,x_final,n) #Feito
    h=(x_final-x_inicial)/n
    
    #método de Euler ou método de Euler modificado (próxima aula)
    
    
    return valor_da_função_no_x_final #também podemos retorna a função completa
end

In [ ]:
function resolver_PVC(segunda_derivada(x,y), x_inicial, y_inicial, x_final, y_final,n)  #Não dá pra fazer genericamente
        A,b=#usuário monta um sistema linear usando diferenças finitas(segunda_derivada(x,y), x_inicial, y_inicial, x_final, y_final,n)
        y = resolver_um_sistema(A,b)
        return y

In [ ]:
function resolver_um_sistema(A,b) #Ainda vamos fazer
    x=A\b #função do Julia , ainda vamos abrir essa caixa
    return x
end

In [ ]:
function interpolação_coeficientes(pontos_x,pontos_y, grau)
    V=matrix_de_vandermonde(pontos_x,grau)
    coeficientes=resolver_um_sistema(V,pontos_y)
    return coeficientes
end

In [ ]:
function matrix_de_vandermonde(pontos_x,grau) #Feito
    return matrix_de_vandermonde
end

In [ ]:
function interpolação_função(pontos_x,pontos_y, grau)
    polinômio=lagrange(pontos_x,pontos_y,grau)
    return polinômio
end

In [ ]:
function lagrange(pontos_x,pontos_y,grau)  #TODO
    
    return polinômio
end

In [ ]:
function aplica(coeficientes, valor_de_x) #Hashi
    return valor_do_polinômio_no_x
end

In [ ]:
function regressão_coeficientes_lineares(pontos_x,pontos_y, grau) #Feito
    V=matrix_de_vandermonde(pontos_x,grau)  #matrix alta #muitas linhas e poucas colunas 
    coeficientes=resolver_um_sistema_por_minimos_quadrados(V,pontos_y)
    return coeficientes
end

In [ ]:
function regressão_função_exponencial(pontos_x,pontos_y, grau) #coeficientes não lineares
    #dicionário ("linearização") é feito manualmente ou por uma tabela
    pontos_y_barra=ln(pontos_y)
    pontos_x_barra=x
    
    #resolver regresssão com coeficientes lineares no "mundo barra"
    regressão_coeficientes_lineares(pontos_x_barra,pontos_y_barra, grau)
    
    #voltar para o mundo original
    coeficientes=e^(coeficientes_barra)
    
    return coeficientes
end

In [ ]:
function regressão_função_outra_funçao(pontos_x,pontos_y, grau) #coeficientes não lineares
    #dicionário ("linearização") é feito manualmente ou por uma tabela
    ??????????????
    
    #resolver regresssão com coeficientes lineares no "mundo barra"
    regressão_coeficientes_lineares(pontos_x_barra,pontos_y_barra, grau)
    
    #voltar para o mundo original
    ???????????????????????
    
    return coeficientes
end

In [ ]:
function resolver_um_sistema_por_minimos_quadrados(A,b) #feito
    x=resolver_um_sistema(A'*A,A'*b)
    return x
end

In [ ]:
function integração(função,a,b,n) 
    
    return 
end

In [ ]:
function regra_do_retângulo(função,a,b,n) #próxima lista
    
    #interpolação constante
    return integral
end

In [ ]:
function regra_do_trapézio(função,a,b,n) #próxima lista
    
    #interpolação linear
    return integral
end

In [ ]:
function regra_de_simpson(função,a,b,n) #vídeo do Abel
    
    #interpolação quadrática
    return integral
end

In [ ]:
function maximar_uma_função(função,derivada) #muito difícil!!!!!!!
    return max_global
end

In [ ]:
function achar_um_ponto_crítico_de_uma_função(derivada_da_função(x),intervalo) #feito
    ponto_crítico=bisseção(derivada(x),intervalo) #ou newton ou ponto fixo...
    return ponto_crítico
end

In [ ]:
function achar_um_ponto_crítico_de_uma_função_de_2_variávies(gradiente_da_função(x),intervalo) #desafio
    ponto_crítico=gradiente_descendente
    return ponto_crítico
end

In [ ]:
function langrage_2D()
    
end